In [551]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
import numpy as np
from collections import OrderedDict, defaultdict
import scipy.stats as sts
from scipy.stats import ks_2samp, kstest, gamma

In [543]:
df = pd.read_excel(r'Prioksk_Air_in.xlsx')
df.fillna(value=np.nan, inplace=True)
df.drop(df.loc[:,'NO':].columns, axis=1, inplace=True)
df.drop(df.loc[:,'ST':'DU'].columns, axis=1, inplace=True)
df.dropna(subset=['YY','MM', 'DD'], inplace=True)
#приведем столбцы с датой к int, сбросим индексы ввиду манипуляций с удалениями, чтобы навести порядок
df.loc[:, 'YY':'DD'] = df.loc[:, 'YY':'DD'].astype(int)
df.reset_index(drop=True, inplace=True)
for i, x in enumerate(df['YY']):
    df.loc[i, 'YY'] = 1900 + x if x >= 83 else 2000 + x
date_column, index_error = [], []
for i in range(df.shape[0]):
    try:
        date_column.append(datetime.date(df.iloc[i,0],df.iloc[i,1],df.iloc[i,2]))
    except ValueError:
        index_error.append(i)
df.drop(index_error, axis = 0, inplace=True)
df.reset_index(drop=True, inplace=True)
#приведение необходимых параметров к типу числовым типам(избавление от типа object)
df = df.apply(pd.to_numeric, errors='coerce')
df.insert(0, 'Time', date_column)
winter_data = df[df['MM'].isin([12, 1, 2])].reset_index(drop=True)
spring_data = df[df['MM'].isin([3, 4, 5])].reset_index(drop=True)
summer_data = df[df['MM'].isin([6, 7, 8])].reset_index(drop=True)
autumn_data = df[df['MM'].isin([9, 10, 11])].reset_index(drop=True)
data_total = df

In [544]:
def ExtractNotNullData(data, element):
    notnulldata = OrderedDict()
    for i in range(data.shape[0]):
        if np.isnan(data.loc[i, element]) == False:
            notnulldata[data.loc[i,'Time']] = data.loc[i, element] 
    return notnulldata

def CreateDataBySeasons(element):
    return {'all':ExtractNotNullData(data_total, element), 'winter':ExtractNotNullData(winter_data, element),
           'spring':ExtractNotNullData(spring_data,element), 'summer':ExtractNotNullData(summer_data, element),
           'autumn':ExtractNotNullData(autumn_data, element)}

def ExtractValues(data):
    return {'all':np.array(list(data['all'].values())),'winter':np.array(list(data['winter'].values())),
           'spring':np.array(list(data['spring'].values())), 'summer':np.array(list(data['summer'].values())),
           'autumn':np.array(list(data['autumn'].values()))}

In [545]:
#В случае Гамма распределения
def ScaleShapeGamma(arr):
    scale = arr.var()/arr.mean()
    shape = arr.mean()**2/arr.var()
    loc = 0
    return shape, loc, scale

def ScipyFitGamma(values):
    return sts.gamma.fit(values, floc=0)

In [546]:
TSP = CreateDataBySeasons('TSP')
TSP_values = ExtractValues(TSP)

In [563]:
all_dates = list(TSP['all'].keys())
[x for x in all_dates if x.year==2010 and x.month==1]

[datetime.date(2010, 1, 2),
 datetime.date(2010, 1, 5),
 datetime.date(2010, 1, 6),
 datetime.date(2010, 1, 8),
 datetime.date(2010, 1, 11),
 datetime.date(2010, 1, 12),
 datetime.date(2010, 1, 14),
 datetime.date(2010, 1, 17),
 datetime.date(2010, 1, 18),
 datetime.date(2010, 1, 20),
 datetime.date(2010, 1, 23),
 datetime.date(2010, 1, 24),
 datetime.date(2010, 1, 26),
 datetime.date(2010, 1, 29),
 datetime.date(2010, 1, 30),
 datetime.date(2010, 1, 31)]

In [573]:
#TSP['all'], dates
#даты подбирал вручную с помощью визуаьного анализа и кода типа такого
#[x for x in all_dates if x.year==2010 and x.month==1] 
dates = [datetime.date(1996,1,1), datetime.date(2010,1,2)]
dates.insert(0, all_dates[0])
dates.append(all_dates[-1])

all_dates = list(TSP['all'].keys())
all_values = list(TSP['all'].values())

divided_data = {}

for i in range(1, len(dates)):
    start_ix = all_dates.index(dates[i-1])
    end_ix = all_dates.index(dates[i])
    key = 'part_' + str(i)
    parts_dict = OrderedDict()
    for j in range(start_ix, end_ix):
        parts_dict[all_dates[j]] = all_values[j]
    divided_data[key] = parts_dict
    
divided_data['part_'+str(len(dates)-1)][all_dates[-1]] = all_values[-1]

### Хи квадрат

In [472]:
from scipy.stats import chisquare, power_divergence, anderson, anderson_ksamp

In [41]:
values = {'winter':np.array(list(TSP_values['winter'])),'spring':np.array(list(TSP_values['spring'])),
          'summer':np.array(list(TSP_values['summer'])),'autumn':np.array(list(TSP_values['autumn']))}

In [76]:
x = np.linspace(0, max(values['winter']),len(values['winter']))
sh1, loc1, sc1 = ScipyFitGamma(values['winter'])
y1 = sts.gamma.pdf(x, sh1, loc1, sc1)
sh2, loc2, sc2 = ScaleShapeGamma(values['winter'])
y2 = sts.gamma.pdf(x, sh2, loc2, sc2)

In [485]:
(sh1,loc1,sc1), (sh2,loc2,sc2)

((1.7280372198753822, 0, 13.417576276029884),
 (1.6606048971087675, 0, 13.962424924715794))

In [531]:
val=values['spring']
param = gamma.fit(val,floc=0)
param

(2.055341741595502, 0, 23.625944935481385)

In [532]:
RuleStugress(len(val))

11

In [533]:
histo, bin_edges = np.histogram(val, bins=RuleStugress(len(val)), density=False)
n_bins = len(bin_edges) - 1
n = sum(histo)
observed_values = histo / n
cdf = gamma.cdf(bin_edges, a=param[0], loc=param[1],scale=param[2])
expected_values = np.diff(cdf)
chi_stat = 0
for i in range(n_bins):
    chi_stat += np.power(observed_values[i] - expected_values[i], 2, dtype=np.float) / expected_values[i]
print(chi_stat*n)

148461.9354314234


In [524]:
#для проверки на R
#вычисляет адекватнее гораздо, p-value, куда более приемлимые
dic = {'x':histo,'p':expected_values}
df = pd.DataFrame(dic, columns=['x', 'p'])
df.to_csv('/Users/Barnett/RProjects/PracDZ/summerforchi2.csv', index=False)

In [509]:
power_divergence(observed_values, expected_values, lambda_=1,ddof=3)

Power_divergenceResult(statistic=0.01343819937519059, pvalue=0.9999999978734517)

In [511]:
def RuleStugress(n):
    return int(1 + np.floor(np.log2(n)))
train=val
sh, loc, sc = ScipyFitGamma(train)
param = ScipyFitGamma(train)
#если рассматривать число элементов и ожидаемое число элементов в интервале
histo, bin_edges = np.histogram(test, bins=RuleStugress(len(val)), density=False)
n_bins = len(bin_edges) - 1
f_ops = histo
cdf = gamma.cdf(bin_edges, *param)
f_exp = len(val) * np.diff(cdf)
print(chisquare(f_ops, f_exp, ddof=len(param)))


Power_divergenceResult(statistic=772.6718977421388, pvalue=1.4624397509636906e-162)


### KS Test

In [260]:
def ScipyFitGamma(values):
    return sts.gamma.fit(values, floc=0)

In [384]:
def KSTest(values, distr):
    if distr == 'gamma':
        sh, loc, sc = ScipyFitGamma(values)
        print (kstest(values, 'gamma', args=(sh,loc,sc)))
        return

In [512]:
KSTest(values['winter'], 'gamma')
KSTest(values['spring'], 'gamma')
KSTest(values['summer'], 'gamma')
KSTest(values['autumn'], 'gamma')

KstestResult(statistic=0.03163425187639102, pvalue=0.07945696755153138)
KstestResult(statistic=0.030519259062621207, pvalue=0.05308409454965937)
KstestResult(statistic=0.03366147343686643, pvalue=0.025543400701630564)
KstestResult(statistic=0.0333641694805242, pvalue=0.049972164400725015)


In [535]:
#два эквивалентных варианта KS теста
sh, loc, sc = ScipyFitGamma(values['summer'])

print(kstest(values['summer'], 'gamma', args=(sh,loc,sc)))

fdist = gamma(sh,loc,sc)
kstest(values['summer'], fdist.cdf)

KstestResult(statistic=0.03366147343686643, pvalue=0.025543400701630564)


KstestResult(statistic=0.03366147343686643, pvalue=0.025543400701630564)

In [287]:
from sklearn.model_selection import train_test_split

In [495]:
train, test= train_test_split(values['winter'], test_size=0.3, random_state=42)
sh, loc, sc = ScipyFitGamma(train)
kstest(test, 'gamma', args=(sh,loc,sc))

KstestResult(statistic=0.03864523038027712, pvalue=0.4634126838114338)

In [510]:
param = ScipyFitGamma(values['winter'])
fdist = gamma(param[0],param[1],param[2])
val = sorted(values['winter'])
val=np.array(val)
anderson_ksamp(val.reshape(-1,1), fdist.cdf)

/Users/Barnett/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: p-value capped: true value larger than 0.25
  """


Anderson_ksampResult(statistic=5.75852681788531e-07, critical_values=array([0.66880938, 1.28705937, 1.66172375, 1.98848063, 2.3713025 ,
       2.63188437, 3.17527875]), significance_level=0.25)

In [541]:
param=ScipyFitGamma(values['winter'])
fdist=gamma(param[0],param[1],param[2])
y = fdist.rvs(size=100)
ks_2samp(values['summer'], y)

Ks_2sampResult(statistic=0.4357157784743992, pvalue=1.6027302680681622e-16)